In [ ]:
import webbrowser
import time
from selenium import webdriver
import cv2
import numpy as np
from imgaug import augmenters as iaa
import os

In [ ]:
#vowels-2309 to 2401
#consonants1-2325 to 2361
#consonants2-2392 to 2399
#numbers 2406 to 2415
#matras 2366 to 2381
#anuswaras 2305 2306 2307

In [ ]:
vowel_start=2309
vowel_end=2324
con1_start=2329 #2325
con1_end=2361
con2_start=2392
con2_end=2399
matra_start=2377 #2366
matra_end=2381
anu_start=2305
anu_end=2307
num_start=2406
num_end=2415

In [ ]:
#each character image will have size 80*80
big_value=5000
char_img_size=80

In [ ]:
#The directory contains ttf font files for Devnagiri fonts
font_list=os.listdir("./fonts/")
print(len(font_list))
print(font_list)

In [ ]:
#text in the generate.html file
font_face=""
for i,j in enumerate(font_list):
    a="@font-face { font-family: %s; src: url('./fonts/%s')} "%("'"+str(i)+"'",j)
    font_face=font_face+a
start_html="<html> <head> <style>"
end_html="</font></p> </body> </html>"
mid_html="</style> </head> <body> <p><font size="+str(15)+">"


In [ ]:
#Augment the different font syled character images 
def Augment_Main(images,path):
        image_aug=Augment1(images)
        img1=np.concatenate((images,image_aug),axis=0)
        image_aug_p1=Augment2(img1)
        image_aug_p2=Augment3(img1)
        img3=np.concatenate((img1,image_aug_p1,image_aug_p2),axis=0)
        image_aug=Augment4(img3)    
        img4=np.concatenate((img3,image_aug),axis=0)
        image_aug_p3=Augment6(img4)
        img5=np.concatenate((img4,image_aug_p3),axis=0)
        image_aug_p4=Augment7(img5)
        img6=np.concatenate((img5,image_aug_p4),axis=0)
        for i in range(len(img6)):
            cv2.imwrite(path+str(i)+".jpg",img6[i])

In [ ]:
#rotate
def Augment1(images):
    aug = iaa.Sequential([
    iaa.Affine(rotate=(-20,20),mode="constant",cval=255)])
    image_aug = aug.augment_images(images)
    print(image_aug)
    return image_aug

In [ ]:
#blur
def Augment2(images):
    print(images[0].shape)
    aug=iaa.OneOf([
                iaa.GaussianBlur((0, 3.0)),
                iaa.AverageBlur(k=(2, 7)),
                iaa.MedianBlur(k=(3, 11))])
    image_aug = aug.augment_images(images)
    return image_aug

In [ ]:
#Either blur or sharpen
def Augment3(images):
    
    aug=iaa.OneOf([
                iaa.Sharpen(alpha=1.0, lightness=(0.75, 2)),
                iaa.Emboss(alpha=1.0, strength=(1.5, 2.0))])
    image_aug = aug.augment_images(images)
    return image_aug

In [ ]:
#insert Gaussian noise
def Augment4(images):
    aug=iaa.Sequential(
        iaa.AdditiveGaussianNoise(scale=0.1*255)
                )
    image_aug = aug.augment_images(images)
    return image_aug


In [ ]:
#super pixel
def Augment5(images):    
    aug=iaa.Sequential(
        iaa.Superpixels(p_replace=0.5, n_segments=128)
                )
    image_aug = aug.augment_images(images)
    return image_aug

In [ ]:
def Augment6(images): 
    aug=iaa.Sequential(
        iaa.ElasticTransformation(alpha=(1, 2.0), sigma=0.25)
    )
    image_aug = aug.augment_images(images)
    return image_aug

In [ ]:
def Augment7(images):
    aug=iaa.Sequential(
        iaa.PiecewiseAffine(scale=(0.03, 0.05))
    )
    image_aug = aug.augment_images(images)
    return image_aug

In [ ]:
#main driver function to render html and take a screenshot
def generate_images():
    for i in range(con1_start,con1_end+1):
        for r in range(matra_start,matra_end+1):
            if not os.path.exists("./data/"+str(i)+"_"+str(r)):
                os.makedirs("./data/"+str(i)+"_"+str(r))
            images=[]
            for j in range(len(font_list)):

                f=open("generate.html","w")
                select_font="* {font-family: %s}"%("'"+str(j)+"'")
                text=start_html+font_face+select_font+mid_html+chr(i)+chr(r)+end_html
                f.write(text)
                f.close()
                driver = webdriver.Chrome('./chromedriver')  # Optional argument, if not specified will search path.

                driver.get('file:///media/crocoder/New Volume4/Personal/Projects/Manuscript Recognition/generate.html')
                driver.save_screenshot("./data/"+str(i)+"_"+str(r)+"/"+str(j)+".png")
                img=save("./data/"+str(i)+"_"+str(r)+"/"+str(j)+".png",char_img_size)
                images.append(cv2.imread("./data/"+str(i)+"_"+str(r)+"/"+str(j)+".png"))
                print("./data/"+str(i)+"_"+str(r)+"/"+str(j)+".png")
                driver.close()
            print(len(images))
            Augment_Main(images,"./data/"+str(i)+"_"+str(r)+"/")

In [ ]:
#crops the image and saves it
def save(path,char_img_size):
    a=cv2.imread(path,0)
    a[a>127]=255
    a[a<=127]=0
    w=np.argmin(a,axis=1)
    w[w==0]=5000
    left=min(w)
    #print(left)
    h=np.argmin(a,axis=0)
    h[h==0]=5000
    top=min(h)
    a=np.rot90(a)
    #print(top)
    h=np.argmin(a,axis=0)
    h[h==0]=5000
    right=min(h)
    right=a.shape[0]-right
    #print(right)
    a=np.rot90(a)
    w=np.argmin(a,axis=0)
    w[w==0]=5000
    bottom=min(w)
    bottom=a.shape[0]-bottom
    #print(bottom)

    a=cv2.imread(path,0)
    a=np.pad(a[top-1:bottom+1,left-1:right+1],(1,1),'constant',constant_values=(255))
    x=int((char_img_size-a.shape[0])/2)
    y=int((char_img_size-a.shape[1])/2)
    img=np.full((char_img_size,char_img_size),255)
    img[x:x+a.shape[0],y:y+a.shape[1]]=a   
    print(img.shape)
    cv2.imwrite(path,img)
    return img

In [ ]:
generate_images()